# Italian Coffee Services

## All imports needed

In [29]:
##API Related
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
## Analysis Related
import codecs
import pandas as pd
import json
import requests
## Machine Learning Related
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
## Map Related
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import numpy as np

## Import data from Wikipedia contains zip codes, borough and neighborhood

In [31]:
response = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html = response.read()
soup = BeautifulSoup(html)
tabulka = soup.find("table", {"class" : "wikitable sortable"})
matrix = []
count = 0
for row in tabulka.findAll('tr'):
    
    ##REMOVE THE FIRST RECORD, SINCE IT IS DIFFERENT
    if (count == 0):
        count = count+1
    else:
        ##CUT ALL EXCEPT 'TD' DATA
        col = row.findAll('td')
        count = count+1
        records = []
        ##CLEAN 'TD' DATA AND ADD IT TO LIST
        records.append(col[0].string.strip())
        records.append(col[1].string.strip())
        records.append(col[2].text.strip())
        ##ADD RECORD TO LIST
        matrix.append(records)
## CREATE DATA FRAME ACCORDING TO MATRIX (LIST OF LIST)
df2 = pd.DataFrame(matrix, columns =['Postcode', 'Borough', 'Neighborhood'])
## FILTER DATA BOROUGH 1= NOT ASSIGNED
df = df2[df2.Borough != 'Not assigned']
## CHANGE NEIGHBORHOOD NAME IF IT COMES WITH A VALUE OF NOT ASSIGNED AS THE NAME OF THE BOROUGH
df.loc[df.Neighborhood == 'Not assigned', ['Neighborhood']] = df.Borough
## GROUP AND ADD THE COMMA BETWEEN NEIGHBORS
df3 =[]
df3 = df.groupby('Postcode').agg({'Borough':'first',
                                'Neighborhood': ', '.join})
df3 = df3.sort_values(by=['Postcode'])
df3.head()

C:\Users\MainUser\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\MainUser\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Borough,Neighborhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


## Import data from .csv file. Coordinates for zip codes.

In [33]:
path_csv = "C:/Users/MainUser/Downloads/Geospatial_Coordinates.csv"
df_2 = pd.read_csv(path_csv, header=0, index_col=0)
print(df_2.head())
df_merge_difkey = pd.merge(df3, df_2, left_on=df3.index, right_on='Postal Code')


              Latitude  Longitude
Postal Code                      
M1B          43.806686 -79.194353
M1C          43.784535 -79.160497
M1E          43.763573 -79.188711
M1G          43.770992 -79.216917
M1H          43.773136 -79.239476


## Get coordinates from Toronto

In [34]:
address = 'Toronto'
neighborhoods = df_merge_difkey
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Call Foursquare API to explore venues around Toronto

In [35]:
CLIENT_ID = 'TMS0A4IFDUWJ5O1ABGW0M0XPMRIT4CSN40I2YOQSM4E2JII5' # your Foursquare ID
CLIENT_SECRET = '3CNDXIKQWSAOO4HG4UIRLMLUOGGUMU4COQB5RKCXQMGTICGJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
limit = 100
radius = 500
neighborhood_latitude= latitude
neighborhood_longitude = longitude
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    limit)
url
results = requests.get(url).json()

## Method to find all venues and their category in Toronto

In [22]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head(50)

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Textile Museum of Canada,Art Museum,43.654396,-79.386500
2,Japango,Sushi Restaurant,43.655268,-79.385165
3,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501
4,Cafe Plenty,Café,43.654571,-79.389450
5,Poke Guys,Poke Place,43.654895,-79.385052
6,Tsujiri,Tea Room,43.655374,-79.385354
7,Rolltation,Japanese Restaurant,43.654918,-79.387424
8,Manpuku まんぷく,Japanese Restaurant,43.653612,-79.390613
9,Karine's,Breakfast Spot,43.653699,-79.390743


## Get Downtown Toronto venues

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Get neighborhoods related to the venues according to the coordinates.

In [12]:
neighborhoods_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods ['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie


In [24]:
print(neighborhoods_venues.shape)
neighborhoods_venues.head()

(1287, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown, St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


In [16]:
print('There are {} uniques categories.'.format(len(neighborhoods_venues['Venue Category'].unique())))
neighborhoods_venues.groupby('Neighborhood').count()

There are 207 uniques categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,88,88,88,88,88,88
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15
Church and Wellesley,87,87,87,87,87,87
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


## Group the data(CREATE CLUSTERS), check which are the main venues in each neighborhood in Downtown Toronto

In [20]:
neighborhoods_onehot = pd.get_dummies(neighborhoods_venues[['Venue Category']], prefix="", prefix_sep="")
neighborhoods_onehot['Neighborhood'] = neighborhoods_venues['Neighborhood'] 
fixed_columns = [neighborhoods_onehot.columns[-1]] + list(neighborhoods_onehot.columns[:-1])
neighborhoods_onehot = neighborhoods_onehot[fixed_columns]
neighborhoods_grouped = neighborhoods_onehot.groupby('Neighborhood').mean().reset_index()
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted1 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted1['Neighborhood'] = neighborhoods_grouped['Neighborhood']

for ind in np.arange(neighborhoods_grouped.shape[0]):
    neighborhoods_venues_sorted1.iloc[ind, 1:] = return_most_common_venues(neighborhoods_grouped.iloc[ind, :], num_top_venues)

kclusters = 5
neighborhoods_grouped_clustering = neighborhoods_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhoods_grouped_clustering)
neighborhoods_venues_sorted1.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_merged = neighborhoods
neighborhoods_merged = neighborhoods_merged.join(neighborhoods_venues_sorted1.set_index('Neighborhood'), on='Neighborhood')
neighborhoods_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Women's Store,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0,Coffee Shop,Restaurant,Park,Pub,Café,Bakery,Italian Restaurant,Pizza Place,Breakfast Spot,Butcher
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Café,Pub,Hotel,Fast Food Restaurant,Gym
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Breakfast Spot,Restaurant,Theater,Café,Mexican Restaurant,Electronics Store
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Tea Room,Italian Restaurant,Bubble Tea Shop,Pizza Place,Japanese Restaurant


## Use the map to help visualize the position of the new store according to the clusters

In [21]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_merged['Latitude'], neighborhoods_merged['Longitude'], neighborhoods_merged['Neighborhood'], neighborhoods_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## According to our cluster and to the information we got, the store should open in Downtown Toronto in the Rosedale neighborhood. It is close to other coffee shops but not on top of them.